In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的端到端ML：MLOps阶段6： 使用Vertex AI批处理预测开始定制文本模型

<table align=left>
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_custom_text_model_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_custom_text_model_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_custom_text_model_batch.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>        
</table>
<br/><br/><br/>

## 概览

本教程演示了如何从自定义文本模型中进行批量预测。

### 目标

在本教程中，您将学习如何在自定义文本模型中使用`Vertex AI Batch Prediction`。

本教程使用以下谷歌云 ML 服务和资源：

- `Vertex AI Batch Prediction`
- `Vertex AI 模型`

执行的步骤包括：

- 下载一个预训练的 TensorFlow RNN 模型。
- 将预训练模型上传为`Vertex AI 模型`资源。
- 对`模型`资源进行批量预测，以JSONL格式。

在使用批量预测和在线预测之间有一个关键区别：

* 预测服务：为整个实例集（即一个或多个数据项）进行即时预测，并实时返回结果。

* 批处理预测服务：对整个实例集进行排队（批）预测，并在准备就绪时将结果存储在 Cloud Storage 存储桶中。

### 模型

本教程中使用的模型是来自 TensorFlow Hub 的预训练的[通用句子编码器](https://tfhub.dev/google/universal-sentence-encoder/4)。该模型接收一个文本句子，并输出一个长度为512的向量嵌入。

### 费用

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage
定价](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

## 安装

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade tensorflow $USER_FLAG -q
! pip3 install --upgrade tensorflow-hub $USER_FLAG -q

### 重新启动内核

在安装额外的包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果有选择的话，请确保在 GPU 运行时中运行本笔记本。在 Colab 中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的 Google Cloud 项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建账户时，您将获得 $300 的免费信用额度，可用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI API、Compute Engine API 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您是在本地运行此笔记本，您需要安装 [Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 将以 `!` 为前缀的行作为 shell 命令运行，并会解析以 `$` 为前缀的 Python 变量。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改`REGION`变量，该变量用于本笔记本的其余操作。以下是Vertex AI支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法在Vertex AI中使用多区域存储桶进行训练。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果你正在进行现场教程，你可能正在使用一个共享的测试账号或项目。为了避免用户在创建资源时发生名称冲突，你可以为每个实例会话创建一个UUID，并将其附加到在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经通过身份验证。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证以通过oAuth进行帐户认证。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务账号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入名称，然后点击**创建**。

在**授予此服务帐号对项目的访问权限**部分，点击角色下拉列表。在筛选框中输入"Vertex"，选择**Vertex管理员**。在筛选框中输入"Storage对象管理员"，选择**Storage对象管理员**。

点击创建。包含您密钥的JSON文件将下载到您的本地环境中。

在下面的单元格中将您的服务帐号密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪个笔记本环境，以下步骤都是必需的。**

当您在 Python 中初始化 Vertex AI SDK 时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源相关数据在会话之间保留的地方。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有您的存储桶尚不存在：才运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最终，通过检查您的云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库和定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
import tensorflow as tf
import tensorflow_hub as hub

初始化Vertex AI SDK用于Python

为您的项目和相应的存储桶初始化Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为预测设置硬件加速器。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持 GPU 的容器映像以及分配给虚拟机实例 (VM) 的 GPU 数量。例如，要使用一个支持 GPU 的容器映像，每个 VM 分配 4 个 Nvidia Telsa K80 GPU，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)


否则，指定`(None, None)`以使用一个在 CPU 上运行的容器映像。

了解更多关于[您所在地区的硬件加速器支持信息](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于预测的预构建 Docker 容器镜像。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置您将用于预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`：每个虚拟CPU 3.75GB内存。
     - `n1-highmem`：每个虚拟CPU 6.5GB内存
     - `n1-highcpu`：每个虚拟CPU 0.9GB内存
 - `vCPUs`：\ [2, 4, 8, 16, 32, 64, 96\] 的数量

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

从TensorFlow Hub获取预训练的编码器嵌入模型

为了演示目的，本教程使用了从TensorFlow Hub（TFHub）获取的预训练的通用句子编码器（USE）编码器模型，然后将其上传到“Vertex AI Model”资源中。一旦您拥有了“Vertex AI Model”资源，该模型就可以部署到“Vertex AI Endpoint”资源中。

### 下载预训练的编码器模型

首先，您需要从TensorFlow Hub下载预训练的通用句子编码器模型。该编码器作为一个TF.Keras层。在本示例中，为了完成模型，您将使用下载的TFHub编码器创建一个“Sequential”模型。

In [ ]:
from tensorflow.keras import Input, Sequential

encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4")

tfhub_model = Sequential([Input(shape=(), dtype=tf.string), encoder])

loss = tf.keras.losses.BinaryCrossentropy()
metrics = tf.metrics.BinaryAccuracy()
optimizer = tf.optimizers.Adam()

tfhub_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
tfhub_model.summary()

### 保存模型工件

在这一点上，模型在内存中。接下来，您将模型工件保存到云存储位置。

In [ ]:
! mkdir model
MODEL_DIR = f"{BUCKET_URI}"
tfhub_model.save("model")

! gsutil cp -r model {MODEL_DIR}
! rm -rf model

MODEL_DIR = MODEL_DIR + "/model"

### 将模型工件上传为`Vertex AI Model`资源

首先，您可以使用`upload()`方法将预训练的自定义表格模型工件作为`Vertex AI Model`资源上传，使用以下参数：

- `display_name`：`Model`资源的人类可读名称。
- `artifact_uri`：模型工件的Cloud Storage位置。
- `serving_container_image`：在将模型部署到`Vertex AI Endpoint`资源时要使用的服务容器映像。
- `sync`：是否等待进程完成，还是立即返回（异步）。

In [ ]:
model = aiplatform.Model.upload(
    display_name="use_" + UUID,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=True,
)

print(model)

## 批量预测简介

批量预测提供了离线批处理大量预测请求的能力。资源仅在批处理过程中进行配置，然后在批处理请求完成时取消配置。结果存储在云存储中，与在线预测不同，后者将结果作为HTTP响应数据返回。

批处理作业的输入格式取决于您的模型服务器支持的格式。首先，您的模型服务器中的Web服务器必须支持JSONL格式，Web服务器将其转换为模型输入接口或服务功能接口直接支持的格式。对于批量预测，该JSONL格式称为“pivot”格式。

### 批量预测作业的输入格式

批处理服务器接受以下自定义文本模型的输入格式：

- JSONL
- 文件列表

批处理服务器接受以下自定义文本模型的输出格式：

- JSONL

### Pivot格式

批处理服务器将输入格式转换为“pivot”（JSONL）格式如下：

**JSONL**

每个输入行（请求）应包含一个并且只有一个有效的json值。

    {"values": [1, 2, 3, 4], "key": 1}
    {"values": [5, 6, 7, 8], "key": 2}

批处理服务器使用相同的格式生成pivot数据。生成的pivot数据然后被包装成一个负载请求：

    {"instances": [
      {"values": [1, 2, 3, 4], "key": 1},
      {"values": [5, 6, 7, 8], "key": 2}
    ]}

**CSV**

第一行中的csv标题将始终被忽略。字符串字段需要明确用双引号括起来，否则该行将被丢弃，解析错误消息将被输出到错误文件中。非引号值始终以浮点数进行传输。

    col1,col2,col3
    1,3,"cat1"
    2,4,"cat2"

批处理服务器将每个输入行（请求）转换为JSON数组。

    {"instances": [
     [1.0,3.0,"cat1"],
     [2.0,4.0,"cat2"]
    ]}
    
**BigQuery**

每一行都被转换为一个JSON数组。例如：

    [1.0,3.0,"cat1"]
    [2.0,4.0,"cat2"]
    
批处理服务器使用相同的格式生成pivot数据。生成的pivot数据然后被包装成一个负载请求：

    {"instances": [
     [1.0,3.0,"cat1"],
     [2.0,4.0,"cat2"]
    ]}

**TFRecords**

TFRecord文件中的实例被apache_beam.io.tfrecordio模块读取为二进制。然后将二进制对象序列化为ASCII字符串。预测服务器负责知道解码器以恢复实例。

    {"instances": [
     {"b64","b64EncodedASCIIString"},
     {"b64","b64EncodedASCIIString"}
    ]}

**文件列表**

文件列表格式包含文件列表。在“FileList”文件中，每行指定一个文件路径，指定为云存储位置。

    gs://my-bucket/file1.txt
    gs://my-bucket/file2.txt

批处理服务器将文件读取为二进制。二进制对象被序列化为ASCII字符串。

    {"instances": [
     {"b64","b64EncodedASCIIString"},
     {"b64","b64EncodedASCIIString"}
    ]}

制作包含JSONL输入的批处理输入文件

现在制作一个批处理输入文件，将其存储在本地云存储桶中。在这个例子中，您使用JSONL格式作为输入和输出。对于输入，JSONL文件的布局如下，每个句子单独占一行并用双引号括起来：

"text_string_1"
"text_string_2"

In [ ]:
gcs_input_uri = BUCKET_URI + "/" + "test.jsonl"
with open("test.jsonl", "w") as f:
    data = "the quick brown fox jumps over the lazy dog"
    f.write('"' + data + '"\n')
    data = "one two"
    f.write('"' + data + '"\n')

! gsutil cp test.jsonl {gcs_input_uri}

### 发送批量预测请求

现在您的模型资源已经训练好，您可以通过调用batch_predict()方法进行批量预测，使用以下参数：

- `job_display_name`：批量预测作业的人类可读名称。
- `instances_format`：批量预测请求文件的格式：“jsonl”，“csv”，“bigquery”，“tf-record”，“tf-record-gzip”或“file-list”。
- `prediction_format`：批量预测响应文件的格式：“jsonl”，“csv”，“bigquery”，“tf-record”，“tf-record-gzip”或“file-list”。
- `gcs_source`：一个或多个批请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的云存储位置。
- `sync`：如果设置为True，则调用将在等待异步批处理作业完成时阻塞。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="use_" + UUID,
    instances_format="jsonl",
    predictions_format="jsonl",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    machine_type=DEPLOY_COMPUTE,
    sync=False,
)

print(batch_predict_job)

等待批量预测作业完成

接下来，等待批量作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻止直到批量预测作业完成为止。

In [ ]:
batch_predict_job.wait()

### 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将被写入你在批量预测请求中指定的云存储输出桶中。你可以调用 iter_outputs() 方法来获取生成的带有结果的每个云存储文件的列表。每个文件都以 JSON 格式包含一个或多个预测请求：

- `content`：预测请求。
- `prediction`：预测响应。
 - `ids`：每个预测请求的内部分配的唯一标识符。
 - `displayNames`：每个类别标签的类名。
 - `confidences`：每个类别标签的预测置信度，介于 0 和 1 之间。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

您可以使用`delete()`方法删除批量预测作业。

In [ ]:
batch_predict_job.delete()

使用文件列表输入创建批处理输入文件

现在创建一个批处理输入文件，将其存储在您的本地云存储桶中。在这个示例中，您将使用文件列表格式进行输入。在这种情况下，文件列表文件中的每一行都指向一个文本文件在云存储中的位置。

准备数据以进行批量预测

接下来，您将相同的批处理预测请求实例格式化为文件列表格式。

In [ ]:
with open("test1.txt", "w") as f:
    f.write('"the quick brown fox jumps over the lazy dog"\n')
with open("test2.txt", "w") as f:
    f.write('"search Google or type an Url"\n')

! gsutil cp test1.txt {BUCKET_URI}/test1.txt
! gsutil cp test2.txt {BUCKET_URI}/test2.txt

! rm test1.txt test2.txt

with open("test.txt", "w") as f:
    f.write(f"{BUCKET_URI}/test1.txt\n")
    f.write(f"{BUCKET_URI}/test2.txt\n")

gcs_input_uri = f"{BUCKET_URI}/test.txt"

! gsutil cp test.txt $gcs_input_uri

### 发送批量预测请求

现在您的模型资源已经训练好了，您可以通过调用batch_predict() 方法来进行批量预测，需要指定以下参数：

- `job_display_name`: 批量预测作业的可读名称。
- `instances_format`: 批量预测请求文件的格式："jsonl"、"csv"、"bigquery"、"tf-record"、"tf-record-gzip" 或 "file-list"。
- `prediction_format`: 批量预测响应文件的格式："jsonl"、"csv"、"bigquery"、"tf-record"、"tf-record-gzip" 或 "file-list"。
- `gcs_source`: 包含一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`: 用于存储批量预测结果的云存储位置。
- `sync`: 如果设置为True，则调用将在等待异步批处理作业完成时阻塞。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="use_" + UUID,
    instances_format="file-list",
    predictions_format="jsonl",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    machine_type=DEPLOY_COMPUTE,
    sync=False,
)

print(batch_predict_job)

### 等待批处理预测任务完成

接下来，等待批处理任务完成。另外，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批处理预测任务完成。

In [ ]:
batch_predict_job.wait()

### 获取预测结果

接下来，从完成的批量预测作业中获取结果。

结果将写入您在批量预测请求中指定的云存储输出桶中。您可以调用 iter_outputs() 方法来获取包含结果的每个云存储文件的列表。每个文件都以 JSON 格式包含一个或多个预测请求：

- `content`：预测请求。
- `prediction`：预测响应。
 - `ids`：每个预测请求的内部分配的唯一标识符。
 - `displayNames`：每个类别标签的类名。
 - `confidences`：每个类别标签的预测置信度，介于 0 和 1 之间。

In [ ]:
import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除您用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除您在此教程中创建的各个资源：

In [ ]:
delete_bucket = True
delete_model = True
delete_batch_job = True

if delete_model:
    try:
        model.delete()
    except Exception as e:
        print(e)
if delete_batch_job:
    batch_prediction_job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}